In [2]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm 

In [3]:
DATA_ROOT = Path("/mnt/HDD_1/FDG/LungCancer_Subtyping/data/nifti")
LUNG_SEG_ROOT = Path("/mnt/HDD_1/FDG/LungCancer_Subtyping/data/processed_yaobo/lung_seg")
EXTERNAL_METADATA_ROOT = Path("/mnt/HDD_1/FDG/LungCancer_Subtyping/data/Dicom_header")

OUTPUT_METADATA_PATH = Path("metadata/metadata.csv")

# 原始数据表格构建

In [6]:
# 增量式表格构建方法
def build_base_table(center):
    """构建基础表格：PID-PET-CT路径信息"""
    base_data = []
    
    # 遍历文件夹获取基础的PID和文件路径信息
    center_path = DATA_ROOT / center
    for case in sorted(center_path.iterdir()):
        if case.is_dir():
            pid = str(case.name)  # 确保PID为字符串类型
            
            # 构建文件路径
            pet_path = case / "PET.nii.gz"
            ct_path = case / "CT.nii.gz"
            lung_seg_path = LUNG_SEG_ROOT / center / pid / "lung_seg.nii.gz"
            
            base_data.append({
                'center': center,
                'PID': pid,
                'pet_path': str(pet_path),
                'ct_path': str(ct_path),
                'lung_seg_path': str(lung_seg_path),
                'pet_exists': pet_path.exists(),
                'ct_exists': ct_path.exists(),
                'lung_seg_exists': lung_seg_path.exists()
            })
    
    return pd.DataFrame(base_data)

def merge_suv_ratio(base_df, center):
    """合并SUV_ratio信息"""
    dicom_header_df = pd.read_csv(f'/mnt/HDD_1/FDG/LungCancer_Subtyping/data/Dicom_header/{center}_dicom_header.csv', dtype={'PID': str})
    
    # 只选择需要的列
    suv_data = dicom_header_df[['PID', 'SUV_ratio']].copy()
    
    # 使用outer join保证取并集，即使有新的PID也会包含进来
    merged_df = pd.merge(base_df, suv_data, on='PID', how='outer')
    
    # 对于新增的PID，需要填充center信息
    merged_df['center'] = merged_df['center'].fillna(center)
    
    return merged_df

def merge_pathology(current_df, center):
    """合并病理信息"""
    histo_df = pd.read_excel(f'/mnt/HDD_1/FDG/LungCancer_Subtyping/data/Dicom_header/{center}_histo.xlsx', 
                            sheet_name='Sheet1', dtype={'PID': str})
    
    # 特殊处理：Neimeng数据需要补0
    if center == 'Neimeng_nifti_425':
        histo_df['PID'] = histo_df['PID'].str.zfill(3)
    
    # 只选择需要的列
    pathology_data = histo_df[['PID', 'Pathology']].copy()
    
    # 使用outer join保证取并集
    merged_df = pd.merge(current_df, pathology_data, on='PID', how='outer')
    
    # 对于新增的PID，需要填充center信息
    merged_df['center'] = merged_df['center'].fillna(center)
    
    return merged_df

# 逐步构建总表
print("开始构建增量式总表...")

all_centers_data = []

for data_center in sorted(DATA_ROOT.iterdir()):
    center = data_center.stem
    if center in ["AKH_nifti_637", "Neimeng_nifti_425"]:
        print(f"\n处理中心: {center}")
        
        # 步骤1: 构建基础表格 (PID-PET-CT)
        print("  步骤1: 构建基础表格 (PID-PET-CT)")
        base_table = build_base_table(center)
        print(f"    基础表格包含 {len(base_table)} 个PID")

        # 确保PID为字符串类型并保存base table
        base_table['PID'] = base_table['PID'].astype(str)
        base_table.to_csv(f"metadata/base_table_{center}.csv", index=False)
        
        # 步骤2: 合并SUV_ratio信息
        print("  步骤2: 合并SUV_ratio信息")
        table_with_suv = merge_suv_ratio(base_table, center)
        print(f"    合并后包含 {len(table_with_suv)} 个PID")
        print(f"    有SUV_ratio数据的: {table_with_suv['SUV_ratio'].notna().sum()} 个")
        
        # 步骤3: 合并病理信息
        print("  步骤3: 合并病理信息")
        final_table = merge_pathology(table_with_suv, center)
        print(f"    最终表格包含 {len(final_table)} 个PID")
        print(f"    有病理数据的: {final_table['Pathology'].notna().sum()} 个")
        
        # 显示数据完整性统计
        complete_files = (final_table['pet_exists'].fillna(False) & 
                         final_table['ct_exists'].fillna(False) & 
                         final_table['lung_seg_exists'].fillna(False))
        print(f"    所有文件完整的: {complete_files.sum()} 个")
        
        all_centers_data.append(final_table)

# 合并所有中心的数据
print("\n合并所有中心的数据...")
final_metadata = pd.concat(all_centers_data, ignore_index=True)

print(f"\n最终统计:")
print(f"总PID数: {len(final_metadata)}")
print(f"各中心分布:")
for center in final_metadata['center'].value_counts().index:
    count = final_metadata['center'].value_counts()[center]
    print(f"  {center}: {count}")

print(f"\n数据完整性:")
print(f"有SUV_ratio: {final_metadata['SUV_ratio'].notna().sum()}")
print(f"有Pathology: {final_metadata['Pathology'].notna().sum()}")
print(f"有PET文件: {final_metadata['pet_exists'].sum()}")
print(f"有CT文件: {final_metadata['ct_exists'].sum()}")
print(f"有肺分割文件: {final_metadata['lung_seg_exists'].sum()}")

# 保存结果
OUTPUT_METADATA_PATH.parent.mkdir(parents=True, exist_ok=True)
# 确保PID为字符串格式
final_metadata['PID'] = final_metadata['PID'].astype(str)
final_metadata.to_csv(OUTPUT_METADATA_PATH, index=False)
print(f"\n数据已保存到: {OUTPUT_METADATA_PATH}")

# 显示前几行
print("\n前5行数据:")
print(final_metadata.head())

开始构建增量式总表...

处理中心: AKH_nifti_637
  步骤1: 构建基础表格 (PID-PET-CT)
    基础表格包含 637 个PID
  步骤2: 合并SUV_ratio信息
    合并后包含 637 个PID
    有SUV_ratio数据的: 636 个
  步骤3: 合并病理信息


    最终表格包含 638 个PID
    有病理数据的: 635 个
    所有文件完整的: 633 个

处理中心: Neimeng_nifti_425
  步骤1: 构建基础表格 (PID-PET-CT)
    基础表格包含 422 个PID
  步骤2: 合并SUV_ratio信息
    合并后包含 427 个PID
    有SUV_ratio数据的: 425 个
  步骤3: 合并病理信息
    最终表格包含 428 个PID
    有病理数据的: 428 个
    所有文件完整的: 417 个

合并所有中心的数据...

最终统计:
总PID数: 1066
各中心分布:
  AKH_nifti_637: 638
  Neimeng_nifti_425: 428

数据完整性:
有SUV_ratio: 1061
有Pathology: 1063
有PET文件: 1059
有CT文件: 1060
有肺分割文件: 1050

数据已保存到: metadata/metadata.csv

前5行数据:
          center                         PID  \
0  AKH_nifti_637  ABDALLA-ADEL-AHMED20091023   
1  AKH_nifti_637        ABT-BRIGITTE20160818   
2  AKH_nifti_637        ADAMEK-KARIN20211124   
3  AKH_nifti_637      AHAMER-GERHARD20240213   
4  AKH_nifti_637       AHMED-MOHAMED20230731   

                                            pet_path  \
0  /mnt/HDD_1/FDG/LungCancer_Subtyping/data/nifti...   
1  /mnt/HDD_1/FDG/LungCancer_Subtyping/data/nifti...   
2  /mnt/HDD_1/FDG/LungCancer_Subtyping/data/nifti...   
3  /mnt/HDD_1/FDG

/tmp/ipykernel_675226/3392047984.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  complete_files = (final_table['pet_exists'].fillna(False) &
/tmp/ipykernel_675226/3392047984.py:96: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_table['ct_exists'].fillna(False) &
/tmp/ipykernel_675226/3392047984.py:97: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_down

## 数据筛选

In [4]:
# 数据筛选 - 筛选出符合条件的样本
print("=== 开始数据筛选 ===")

# 读取完整的metadata
if 'final_metadata' not in locals():
    final_metadata = pd.read_csv(OUTPUT_METADATA_PATH, dtype={'PID': str})

print(f"原始数据总数: {len(final_metadata)}")

# 筛选条件1: 三个模态的数据都存在
condition_files_exist = (
    final_metadata['pet_exists'] == True
) & (
    final_metadata['ct_exists'] == True
) & (
    final_metadata['lung_seg_exists'] == True
)

print(f"三个模态数据都存在的样本: {condition_files_exist.sum()}")

# 筛选条件2: SUV_ratio存在
condition_suv_exists = final_metadata['SUV_ratio'].notna()
print(f"有SUV_ratio数据的样本: {condition_suv_exists.sum()}")

# 筛选条件3: Pathology为SCC、ADC、SCLC之一
valid_pathologies = ['SCC', 'ADC', 'SCLC']
condition_pathology_valid = final_metadata['Pathology'].isin(valid_pathologies)
print(f"病理类型为{valid_pathologies}的样本: {condition_pathology_valid.sum()}")

# 查看当前所有的病理类型分布
print(f"\n当前所有病理类型分布:")
pathology_counts = final_metadata['Pathology'].value_counts()
for pathology, count in pathology_counts.items():
    print(f"  {pathology}: {count}")

# 合并所有筛选条件
final_condition = condition_files_exist & condition_suv_exists & condition_pathology_valid

# 应用筛选
filtered_metadata = final_metadata[final_condition].copy()

print(f"\n=== 筛选结果 ===")
print(f"筛选后样本数: {len(filtered_metadata)}")
print(f"筛选保留率: {len(filtered_metadata)/len(final_metadata)*100:.1f}%")

# 查看筛选后各中心的分布
print(f"\n各中心筛选后分布:")
center_distribution = filtered_metadata['center'].value_counts()
for center, count in center_distribution.items():
    print(f"  {center}: {count}")

# 查看筛选后的病理类型分布
print(f"\n筛选后病理类型分布:")
filtered_pathology_counts = filtered_metadata['Pathology'].value_counts()
for pathology, count in filtered_pathology_counts.items():
    print(f"  {pathology}: {count}")

# 验证筛选结果的数据完整性
print(f"\n=== 筛选后数据完整性验证 ===")
print(f"PET文件存在: {filtered_metadata['pet_exists'].sum()}/{len(filtered_metadata)}")
print(f"CT文件存在: {filtered_metadata['ct_exists'].sum()}/{len(filtered_metadata)}")
print(f"肺分割文件存在: {filtered_metadata['lung_seg_exists'].sum()}/{len(filtered_metadata)}")
print(f"SUV_ratio存在: {filtered_metadata['SUV_ratio'].notna().sum()}/{len(filtered_metadata)}")
print(f"Pathology存在: {filtered_metadata['Pathology'].notna().sum()}/{len(filtered_metadata)}")

# 保存筛选后的数据
filtered_output_path = Path("metadata/metadata_filtered.csv")
filtered_metadata.to_csv(filtered_output_path, index=False)
print(f"\n筛选后数据已保存到: {filtered_output_path}")

# 显示前几行
print(f"\n筛选后数据前5行:")
print(filtered_metadata[['center', 'PID', 'SUV_ratio', 'Pathology', 'pet_exists', 'ct_exists', 'lung_seg_exists']].head())

=== 开始数据筛选 ===
原始数据总数: 1066
三个模态数据都存在的样本: 1050
有SUV_ratio数据的样本: 1061
病理类型为['SCC', 'ADC', 'SCLC']的样本: 1043

当前所有病理类型分布:
  ADC: 582
  SCC: 336
  SCLC: 125
  LCNEC: 13
  LCCL: 2
  Sarcoma: 2
  MEC: 1
  ADC+SCC: 1
  LCLC: 1

=== 筛选结果 ===
筛选后样本数: 1030
筛选保留率: 96.6%

各中心筛选后分布:
  AKH_nifti_637: 618
  Neimeng_nifti_425: 412

筛选后病理类型分布:
  ADC: 574
  SCC: 331
  SCLC: 125

=== 筛选后数据完整性验证 ===
PET文件存在: 1030/1030
CT文件存在: 1030/1030
肺分割文件存在: 1030/1030
SUV_ratio存在: 1030/1030
Pathology存在: 1030/1030

筛选后数据已保存到: metadata/metadata_filtered.csv

筛选后数据前5行:
          center                         PID  SUV_ratio Pathology pet_exists  \
0  AKH_nifti_637  ABDALLA-ADEL-AHMED20091023   0.000325       SCC       True   
1  AKH_nifti_637        ABT-BRIGITTE20160818   0.000353       ADC       True   
2  AKH_nifti_637        ADAMEK-KARIN20211124   0.000534       ADC       True   
4  AKH_nifti_637       AHMED-MOHAMED20230731   0.000517       ADC       True   
5  AKH_nifti_637        AHMEDI-NAZIF20230111   0.000578      

In [6]:
# 筛选结果详细统计分析
print("=== 筛选结果详细分析 ===")

# 读取筛选后的数据
filtered_metadata = pd.read_csv("metadata/metadata_filtered.csv", dtype={'PID': str})

# 1. 总体统计
print(f"\n1. 总体统计:")
print(f"   - 筛选前总样本: 1066")
print(f"   - 筛选后总样本: {len(filtered_metadata)}")
print(f"   - 筛选保留率: {len(filtered_metadata)/1066*100:.1f}%")
print(f"   - 被排除样本: {1066-len(filtered_metadata)}")

# 2. 筛选后样本分布统计
print(f"\n2. 筛选后样本分布统计:")

# 总体统计
print(f"\n   总体分布 (总样本: {len(filtered_metadata)}):")
pathology_distribution = filtered_metadata['Pathology'].value_counts()
for pathology in ['ADC', 'SCC', 'SCLC']:  # 按固定顺序显示
    count = pathology_distribution.get(pathology, 0)
    percentage = count / len(filtered_metadata) * 100
    print(f"     {pathology}: {count} ({percentage:.1f}%)")

# 各中心详细统计
print(f"\n   各中心详细分布:")
for center in ['AKH_nifti_637', 'Neimeng_nifti_425']:
    center_data = filtered_metadata[filtered_metadata['center'] == center]
    center_total = len(center_data)
    
    print(f"\n   {center} (总样本: {center_total}):")
    for pathology in ['ADC', 'SCC', 'SCLC']:
        count = len(center_data[center_data['Pathology'] == pathology])
        percentage = count / center_total * 100 if center_total > 0 else 0
        print(f"     {pathology}: {count} ({percentage:.1f}%)")

# 3. 数据分布汇总表格
print(f"\n3. 数据分布汇总表:")
print(f"{'中心':<20} {'总样本':<8} {'ADC':<12} {'SCC':<12} {'SCLC':<12}")
print("-" * 70)

# 各中心统计
for center in ['AKH_nifti_637', 'Neimeng_nifti_425']:
    center_data = filtered_metadata[filtered_metadata['center'] == center]
    total = len(center_data)
    adc_count = len(center_data[center_data['Pathology'] == 'ADC'])
    scc_count = len(center_data[center_data['Pathology'] == 'SCC'])
    sclc_count = len(center_data[center_data['Pathology'] == 'SCLC'])
    
    print(f"{center:<20} {total:<8} {adc_count:<12} {scc_count:<12} {sclc_count:<12}")

# 总计
total_all = len(filtered_metadata)
adc_total = len(filtered_metadata[filtered_metadata['Pathology'] == 'ADC'])
scc_total = len(filtered_metadata[filtered_metadata['Pathology'] == 'SCC'])
sclc_total = len(filtered_metadata[filtered_metadata['Pathology'] == 'SCLC'])

print("-" * 70)
print(f"{'总计':<20} {total_all:<8} {adc_total:<12} {scc_total:<12} {sclc_total:<12}")

# 4. 原始数据对比
print(f"\n4. 筛选前后对比:")
original_metadata = pd.read_csv(OUTPUT_METADATA_PATH, dtype={'PID': str})
print(f"{'中心':<20} {'原始样本':<10} {'筛选后':<10} {'保留率':<10}")
print("-" * 55)

for center in ['AKH_nifti_637', 'Neimeng_nifti_425']:
    original_count = len(original_metadata[original_metadata['center'] == center])
    filtered_count = len(filtered_metadata[filtered_metadata['center'] == center])
    retention_rate = filtered_count / original_count * 100 if original_count > 0 else 0
    
    print(f"{center:<20} {original_count:<10} {filtered_count:<10} {retention_rate:<10.1f}%")

# 总计对比
original_total = len(original_metadata)
filtered_total = len(filtered_metadata)
total_retention = filtered_total / original_total * 100

print("-" * 55)
print(f"{'总计':<20} {original_total:<10} {filtered_total:<10} {total_retention:<10.1f}%")

# 5. SUV_ratio统计
print(f"\n5. SUV_ratio统计:")
suv_stats = filtered_metadata['SUV_ratio'].describe()
print(f"   - 最小值: {suv_stats['min']:.6f}")
print(f"   - 最大值: {suv_stats['max']:.6f}")
print(f"   - 平均值: {suv_stats['mean']:.6f}")
print(f"   - 中位数: {suv_stats['50%']:.6f}")
print(f"   - 标准差: {suv_stats['std']:.6f}")

print(f"\n=== 筛选完成 ===")
print(f"筛选后的metadata已保存为: metadata_filtered.csv")
print(f"可用于后续分析的高质量样本: {len(filtered_metadata)} 个")

=== 筛选结果详细分析 ===

1. 总体统计:
   - 筛选前总样本: 1066
   - 筛选后总样本: 1030
   - 筛选保留率: 96.6%
   - 被排除样本: 36

2. 筛选后样本分布统计:

   总体分布 (总样本: 1030):
     ADC: 574 (55.7%)
     SCC: 331 (32.1%)
     SCLC: 125 (12.1%)

   各中心详细分布:

   AKH_nifti_637 (总样本: 618):
     ADC: 340 (55.0%)
     SCC: 199 (32.2%)
     SCLC: 79 (12.8%)

   Neimeng_nifti_425 (总样本: 412):
     ADC: 234 (56.8%)
     SCC: 132 (32.0%)
     SCLC: 46 (11.2%)

3. 数据分布汇总表:
中心                   总样本      ADC          SCC          SCLC        
----------------------------------------------------------------------
AKH_nifti_637        618      340          199          79          
Neimeng_nifti_425    412      234          132          46          
----------------------------------------------------------------------
总计                   1030     574          331          125         

4. 筛选前后对比:
中心                   原始样本       筛选后        保留率       
-------------------------------------------------------
AKH_nifti_637        638        618   